Se importan los paquetes necesarios

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import sys
from src.cleaning_functions import *

Se importan los datos como pandas DataFrame

In [3]:
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

FileNotFoundError: [Errno 2] No such file or directory: 'data/attacks.csv'

Se exploran los datos

In [ ]:
sharks.shape

In [ ]:
sharks.sample()

In [ ]:
sharks.isna().sum()

Se comprueba que es un DataFrame muy sucio con una gran cantidad de nulos

Se cambia el nombre de todas las columnas del DataFrame para simplificar

In [ ]:
nuevas_columna = {columna:(columna.replace(" ", "_").replace(".", "_").lower() if columna[-1]!=" "
                  else columna.replace(" ", "_").replace(".", "_").lower()[:-1]) for columna in list(sharks.keys())}
sharks.rename(columns=nuevas_columna, inplace=True)

In [ ]:
sharks.sample()

Ahora se va a proceder a la limpieza de los datos.

Lo primero es eliminar todas las filas con datos repetidos

In [ ]:
sharks.drop_duplicates(inplace=True)

A continuación se van a eliminar todas aquellas filas donde todas las celdas sean nulos y todas las columnas donde todos los valores sean nulos

In [ ]:
sharks.dropna(axis=0, how='all', inplace=True)
sharks.dropna(axis=1, how='all', inplace=True)

In [ ]:
sharks.shape

Con este proceso se han limpiado bastantes filas.

In [ ]:
sharks[(sharks["case_number"]=="0")].isna().sum()

Se comprueba que en aquellas columnas donde "case_number" es nulo, todo el resto de columnas salvo "original__order" son nulas también, por lo que no aportan información útil y se van a eliminar.

In [ ]:
sharks.drop(sharks[(sharks["case_number"]=="0")].index,inplace= True)

Viendo la información del DF se ve que en las columnas "unnamed:_22" y "unnamed:_23" casi todos los valores son NaN. Se va a verificar cuales son aquellos valores no nulos y posteriormente eliminar las columnas completas

In [ ]:
sharks[sharks["unnamed:_22"].notna()].shape

In [ ]:
sharks[sharks["unnamed:_22"].notna()].head()

In [ ]:
sharks[sharks["unnamed:_23"].notna()].shape

In [ ]:
sharks[sharks["unnamed:_23"].notna()].head(2)

No hay ningún dato valioso en esas columnas por lo que se van a eliminar

In [ ]:
sharks.drop(columns = ["unnamed:_22", "unnamed:_23"],axis=1,inplace= True)

Ahora se va a estudiar la diferencia entre "case_number"case_number_1" y "case_number_2"

In [ ]:
sharks[sharks["case_number_1"]!=sharks["case_number_2"]].head(21)[["case_number","case_number_1","case_number_2"]]

Viendo la compación de las columnas "case_number_1" y"case_number_2" con "case_number" se ve que la columna "case_number_1" contiene la misma información que la de "case_number_2" salvo en una pequeña cantidad de casos donde la columna "case_number" coincide con "case_number_2" por lo que se asume que la columna "case_number_1" no aporta nada de valor

In [ ]:
sharks.drop(columns = ["case_number_1"],axis=1,inplace= True)

In [ ]:
sharks[sharks["case_number"]!=sharks["case_number_2"]][["case_number","case_number_2","date"]].sample(5)

Se comprueba que las columnas "case_number" y "case_number_2" son iguales salvo en unos casos donde comprobando con la columna date se ve que es la columna "case_number" la que es erronea

In [ ]:
sharks["case_number"]=sharks["case_number_2"]
sharks.drop(columns = ["case_number_2"],axis=1,inplace= True)

In [ ]:
sharks.sample(1)

Hasta aquí se han limpiado del DataFrame todas aquellas filas con todos los valores NaN y todas aquellas columnas que no aportan nada de información útil, a continuación se va a exportar el DataFrame modificado a un .csv

In [ ]:
sharks.to_csv("data/sharks_limpio_raw.csv",index=False)

In [ ]:
sharks = pd.read_csv("data/sharks_limpio_raw.csv",encoding = "ISO-8859-1")

Se van a eliminar todas aquellas columnas que no se van a usar para sacar conclusiones

In [ ]:
sharks.drop(columns = ["investigator_or_source","pdf","href_formula","href","case_number",
                       "original_order","name","area","location"],axis=1,inplace= True)

Ahora se va a hacer una limpieza de cada columna útil, para ello se van a utilizar las funciones del archivo de funciones

In [ ]:
sharks["sex"] = sharks["sex"].apply(limpieza_sex)
sharks["age"] = sharks["age"].apply(limpieza_age)
sharks["year"] = sharks["year"].apply(limpieza_year)
sharks["fatal_(y/n)"] = sharks["fatal_(y/n)"].apply(limpieza_fatal)
sharks["hour"] = sharks["time"].apply(limpieza_time)
sharks.drop(columns = ["time"],axis=1,inplace= True)
sharks["type"] = sharks["type"].apply(limpieza_type)
sharks["months_code"] = sharks["date"].apply(limpieza_meses)
sharks["day"] = sharks["date"].apply(limpieza_dias)
sharks.drop(columns = ["date"],axis=1,inplace= True)
sharks["species"] = sharks["species"].apply(limpieza_species)
sharks["injury"] = sharks["injury"].apply(limpieza_injury)
sharks["country"] = sharks["country"].apply(limpieza_country)
sharks["activity"] = sharks["activity"].apply(limpieza_activity)

Se reordena el DataFrame

In [ ]:
sharks = sharks.iloc[:,[0,10,11,9,2,4,5,3,8,1,6,7]]
sharks.sample()

Se eliminan las filas duplicadas y vacias

In [ ]:
sharks.drop_duplicates(inplace=True)
sharks.dropna(axis=0, how='all', inplace=True)
sharks.dropna(axis=1, how='all', inplace=True)

In [ ]:
sharks.shape

Se exporta el .csv limpio

In [ ]:
sharks.to_csv("shark_attacks.csv",index=False)